In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [8]:
class RelationNet(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, class_size, num_layers):
        super(RelationNet, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        W = torch.randn((vocab_size, embed_size), requires_grad=True)
        self.embed.weight = nn.Parameter(W)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, bidirectional = True, batch_first = True)
        self.linear = nn.Linear(4*hidden_size, class_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropout = nn.Dropout(0.35)
        
    def forward(self, sentence):
        embeddings = self.embed(sentence)
        hiddens, _ = self.lstm(self.dropout(embeddings))
#         lstm_out = torch.cat((hiddens[:, :, 149], hiddens[:, :, 299]), dim = 1)
        lstm_out = torch.cat((hiddens[:, 0, :], hiddens[:, -1, :]), dim = 1)
        outputs = self.linear(lstm_out)
        outputs = self.softmax(outputs)

        return outputs

In [ ]:
class RelationNet_glove(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, class_size, num_layers):
        super(RelationNet_glove, self).__init__()
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, bidirectional = True, batch_first = True)
        self.linear = nn.Linear(hidden_size*4, class_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropout = nn.Dropout(0.35)
        
    def forward(self, sentence):
        hiddens, _ = self.lstm(self.dropout(sentence))
        lstm_out = torch.cat((hiddens[:, 0, :], hiddens[:, -1, :]), dim = 1)
        outputs = self.linear(lstm_out)
        outputs = self.softmax(outputs)

        return outputs

In [ ]:
class RelationNet_glove_indexed(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, class_size, num_layers):
        super(RelationNet_glove_indexed, self).__init__()
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, bidirectional = True, batch_first = True)
        self.linear = nn.Linear(hidden_size*2, class_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropout = nn.Dropout(0.35)
        
    def forward(self, sentence_index):
        sentence, index = sentence_index[0], sentence_index[1]
        inds = torch.round(index.float().mean(0))
        hiddens, _ = self.lstm(self.dropout(sentence))
        output_forward, output_backward = torch.chunk(hiddens, 2, 2)
        lstm_out = torch.cat((output_forward.unsqueeze(2),output_backward.unsqueeze(2)), dim=2)
        lstm_out, b = torch.max(lstm_out, 2)
        lstm_out = torch.cat((lstm_out[:, int(inds[0]), :], lstm_out[:, int(inds[1]), :]), dim = 1)
        outputs = self.linear(lstm_out)
        outputs = self.softmax(outputs)

        return outputs

In [ ]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    step = checkpoint["step"]
    return modelstep